In [5]:
import pandas as pd
import cx_Oracle
import os

In [7]:
# basic code
intern = cx_Oracle.makedsn('119.147.213.115', '1522', service_name='wktzorcl')
os.chdir('/Users/kai/Desktop/qs/index')
def queryOracle(sql):
    # 创建连接
    conn = cx_Oracle.connect(user='wktz_intern01', password='wktz_intern01', dsn=intern)
    # 创建游标
    curs = conn.cursor()
    # 执行查询
    curs.execute(sql)
    # 返回查询结果
    data = curs.fetchall()
    # 获取列名
    columns = [i[0] for i in curs.description]
    # 将结果转换为DataFrame格式
    df = pd.DataFrame(data, columns=columns)
    # 关闭游标和连接
    curs.close()
    conn.close()
    
    return df

In [15]:
# 1. 获取所有股index代码
sql_get_all_index_code = '''
SELECT distinct(s_info_windcode) FROM WKWD_SYNC.aindexeodprices
'''
indexcodes = queryOracle(sql_get_all_index_code)
indexcodes.to_csv('indexcodes.csv', index=False)

In [12]:
for i in indexcodes['S_INFO_WINDCODE']:
    # print i if i begins with 000
    if i.startswith('000001.S') or i.startswith('399001.S') or i.startswith('399006.S') or i.startswith('000300.S') or i.startswith('000016.S'):
        print(i)

399006.SZ
000016.SH
000001.SH
399001.SZ
000300.SH


In [18]:
sql = '''
    SELECT trade_dt, s_info_windcode FROM WKWD_SYNC.aindexeodprices
    WHERE s_info_windcode IN ('399006.SZ', '000016.SH', '000001.SH', '399001.SZ', '000300.SH')
    AND s_dq_pctchange > 3
'''
index_3 = queryOracle(sql)

In [19]:
# 将index_3中的日期按照不同的s_info_windcode分成不同的csv文件
for i in index_3['S_INFO_WINDCODE'].unique():
    index_3[index_3['S_INFO_WINDCODE'] == i].to_csv(i + '.csv', index=False)